In [1]:

from copy import deepcopy

import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import time

In [2]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [3]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=28 * 28, hidden_size=16):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, 10)
        # self.fc1 = nn.Linear(hidden_size, 10)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc0(x)
        return x

In [5]:
# Dataset and Dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Load dataset and split into train/validation sets
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [6]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0])

In [7]:
hyperparams = {
    "num_epochs": 64,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.7},
    "replace_layers": ["fc0"],
    "threshold": 0.004,
    "min_delta_epoch_replace": 8,
    "window_size": 5,
    "lr": 1e-4,
    "delete_after": 2,    
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 64, metric: AbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc0': 0.7}, replace_layers: ['fc0'], threshold: 0.004, min_delta_epoch_replace: 8, window_size: 5, lr: 0.0001, delete_after: 2"

In [8]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vanyamironov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
run = wandb.init(
    project="self-expanding-nets",
    name=f"trash",
    config=hyperparams
)

In [10]:
criterion = nn.CrossEntropyLoss()
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, criterion, hyperparams)

100%|██████████| 750/750 [00:02<00:00, 314.12it/s]


Epoch 1/64, Train Loss: 1.4805, Val Loss: 0.9977, Val Accuracy: 0.8192


100%|██████████| 750/750 [00:02<00:00, 313.47it/s]


Epoch 2/64, Train Loss: 0.8089, Val Loss: 0.6842, Val Accuracy: 0.8527


100%|██████████| 750/750 [00:02<00:00, 288.25it/s]


...

Epoch 62/64, Train Loss: 0.0337, Val Loss: 0.0859, Val Accuracy: 0.9752


100%|██████████| 750/750 [00:14<00:00, 51.62it/s]


Epoch 63/64, Train Loss: 0.0325, Val Loss: 0.0847, Val Accuracy: 0.9757
Chosen edges: tensor([[   9,    9,    9,    9,    2,    2,    7,    9,    9,    2,    9,    2,
            8,    9,    9,    9,    2,    7,    9,    7,    9,    9,    9,    3,
            9,    7,    9,    9,    9,    2,    3,    9,    9,    9,    2,    3,
            9,    2,    3,    9,    9,    7,    7,    9,    9,    9,    3,    9,
            9,    9,    9],
        [ 851,  885, 1045, 1322, 1421, 1422, 1426, 1426, 1428, 1442, 1451, 1452,
         1452, 1455, 1456, 1468, 1477, 1477, 1477, 1485, 1485, 1486, 1488, 1497,
         1501, 1502, 1502, 1503, 1506, 1510, 1510, 1510, 1511, 1512, 1517, 1517,
         1517, 1531, 1544, 1544, 1548, 1555, 1559, 1559, 1561, 1568, 1572, 1586,
         1593, 1596, 1597]]) 51
51


100%|██████████| 750/750 [00:16<00:00, 45.88it/s]


Epoch 64/64, Train Loss: 0.0311, Val Loss: 0.0848, Val Accuracy: 0.9759
